In [4]:
import time
import pygame
import pygame.midi

In [5]:
def print_midi_info():
    pygame.midi.init()
    for i in range(pygame.midi.get_count()):
        print(pygame.midi.get_device_info(i))
    
    print("Default in: ", pygame.midi.get_default_input_id())
    print("Default out: ", pygame.midi.get_default_output_id())
    pygame.midi.quit()

print_midi_info()

(b'MMSystem', b'Microsoft MIDI Mapper', 0, 1, 0)
(b'MMSystem', b'AKM322', 1, 0, 0)
(b'MMSystem', b'VirtualMIDISynth #1', 0, 1, 0)
(b'MMSystem', b'Microsoft GS Wavetable Synth', 0, 1, 0)
(b'MMSystem', b'AKM322', 0, 1, 0)
Default in:  1
Default out:  0


In [3]:
def take_input():
    pygame.init()
    pygame.fastevent.init() #??????

    event_get = pygame.fastevent.get #??
    event_post = pygame.fastevent.post #??

    pygame.midi.init()
    
    in_id = pygame.midi.get_default_input_id()
    def_o = pygame.midi.get_default_output_id()

    i = pygame.midi.Input(in_id)
    o = pygame.midi.Output(def_o)
    o.set_instrument(0)

    pygame.display.set_mode((1,1))
    cont = True
    note_count = 0

    while note_count < 5:                
        if i.poll():
            midi_events = i.read(1)
            #for list in midi_events:
            #midi_evs = pygame.midi.midis2events(midi_events, i.device_id)       
            print(midi_events)                                                                                                                                                                                                                                                                                                                                                                                                                       
            if midi_events[0][0][0] == 144: #if note on
                o.note_on(midi_events[0][0][1], 120)
            if midi_events[0][0][0] == 128: #if note off
                o.note_off(midi_events[0][0][1], 120)
                note_count += 1  #DOES NOT ACCOUNT FOR CHORDS
        if(note_count == 5):
            time.sleep(1)
            #o.note_on(list[0][0], 127)
            #o.note_on(midi_events[0][1], 64)
            #o.note_on(list[0][2], 64)
            #time.sleep(1)
            #o.note_off(list[0][0], 127)
            #o.note_off(midi_events[0][1], 64)
            #o.note_off(list[0][2], 64)

    i.close()
    o.close()
    pygame.midi.quit()
    pygame.quit()

take_input()

[[[144, 60, 1, 0], 1593]]
[[[128, 60, 0, 0], 1951]]
[[[144, 59, 42, 0], 2100]]
[[[128, 59, 0, 0], 2414]]
[[[144, 57, 46, 0], 2622]]
[[[128, 57, 0, 0], 2930]]
[[[144, 59, 1, 0], 3098]]
[[[128, 59, 0, 0], 3435]]
[[[144, 60, 40, 0], 3583]]
[[[128, 60, 0, 0], 4018]]
